In [4]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.modelspec import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

In [6]:
model_name = "token_transfers"


# Prepare data raeders
data_spec = ModelsDataSpec(models=[model_name],
    root_path_prefix="blockbatch")
blockbatch_request = BlockBatchRequest.build(
    chains=["op"],
    range_spec="@20241118:+1",
    root_paths_to_read=data_spec.input_root_paths,
)
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)

2025-06-17 16:41:38 [debug    ] connecting to GOLDSKY Clickhouse client... counter=001/001 eta=None filename=client.py lineno=56 process=9954
2025-06-17 16:41:38 [info     ] loaded vault from .env file    counter=001/001 eta=None filename=vault.py lineno=32 process=9954
2025-06-17 16:41:38 [debug    ] loaded vault: 28 items         counter=001/001 eta=None filename=vault.py lineno=79 process=9954
2025-06-17 16:41:39 [debug    ] initialized GOLDSKY Clickhouse client. counter=001/001 eta=None filename=client.py lineno=61 process=9954
2025-06-17 16:41:39 [info     ] querying markers at time range min: 2024-11-18 00:00:00, max: 2024-11-19 00:00:00 root_paths=['ingestion/logs_v1', 'ingestion/traces_v1'] filename=request.py lineno=146 process=9954
2025-06-17 16:41:39 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=56 process=9954
2025-06-17 16:41:40 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=61 process=9954
2025-06-17 16:41:40 

In [7]:
logs_view = input_datasets["ingestion/logs_v1"].create_view()
traces_view = input_datasets["ingestion/traces_v1"].create_view()

all_transfers = auxiliary_templates["token_transfers"].to_relation(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": logs_view,
    },
)

native_transfers = auxiliary_templates["native_transfers"].to_relation(
    duckdb_context=ctx,
    template_parameters={
        "raw_traces": traces_view,
    },
)

2025-06-17 16:41:43 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=291 process=9954
2025-06-17 16:41:44 [info     ] created table/view ingestion_logs_v1_view filename=client.py lineno=230 process=9954
2025-06-17 16:41:44 [info     ] duck db size: 12.3KB           filename=client.py lineno=40 process=9954
2025-06-17 16:41:44 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=291 process=9954
2025-06-17 16:41:44 [info     ] created table/view ingestion_traces_v1_view filename=client.py lineno=230 process=9954
2025-06-17 16:41:44 [info     ] duck db size: 12.3KB           filename=client.py lineno=40 process=9954
2025-06-17 16:41:44 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=9954 template=token_transfers
2025-06-17 16:41:44 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=9954 template=native_transfers


In [8]:
df = ctx.client.sql(f"SELECT * FROM {logs_view} as l where l.topic0 LIKE '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef%' LIMIT 10").df()
df.head()

,network,chain_id,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,log_index,address,topics,data,topic0,indexed_args,chain,dt
0,mainnet,10,1731890755,128145989,0x98f6d6cfb9de5b5ccf9c3d9849bc04ab9a2c4725b657...,0x6ba43fabde9f03dded7c56677751114907201a9e4462...,1,0,0xdc6ff44d5d932cbd77b52e5612ba0529dc6226f1,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x00000000000000000000000000000000000000000000...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x000000000000000000000000f89d7b9c864f589bbf5...,op,2024-11-18
1,mainnet,10,1731890755,128145989,0x98f6d6cfb9de5b5ccf9c3d9849bc04ab9a2c4725b657...,0x026e632a5d3a57e54c0f786ae4249b318dcc9ff728f5...,4,1,0x4200000000000000000000000000000000000042,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x00000000000000000000000000000000000000000000...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x0000000000000000000000004dc22588ade05c40338...,op,2024-11-18
2,mainnet,10,1731890755,128145989,0x98f6d6cfb9de5b5ccf9c3d9849bc04ab9a2c4725b657...,0x0f3da35693644abe74388ab718881855a50e147201bb...,6,10,0x4200000000000000000000000000000000000006,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x00000000000000000000000000000000000000000000...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x00000000000000000000000088391365c2259730322...,op,2024-11-18
3,mainnet,10,1731890755,128145989,0x98f6d6cfb9de5b5ccf9c3d9849bc04ab9a2c4725b657...,0x0f3da35693644abe74388ab718881855a50e147201bb...,6,14,0x4200000000000000000000000000000000000042,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x00000000000000000000000000000000000000000000...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x000000000000000000000000442659a6d04b907c879...,op,2024-11-18
4,mainnet,10,1731890755,128145989,0x98f6d6cfb9de5b5ccf9c3d9849bc04ab9a2c4725b657...,0x026e632a5d3a57e54c0f786ae4249b318dcc9ff728f5...,4,2,0x4200000000000000000000000000000000000006,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x00000000000000000000000000000000000000000000...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,[0x000000000000000000000000802b65b5d9016621e66...,op,2024-11-18


In [9]:
erc20_transfers = (
    all_transfers
    .filter("token_id IS NULL")
    .project("* EXCLUDE token_id")
)

df = ctx.client.sql(f"SELECT * FROM erc20_transfers").df()
df.head()

,dt,chain,chain_id,network,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,log_index,contract_address,amount,amount_lossless,from_address,to_address
0,2024-11-18,op,10,mainnet,1731890755,128145989,0x98f6d6cfb9de5b5ccf9c3d9849bc04ab9a2c4725b657...,0x6ba43fabde9f03dded7c56677751114907201a9e4462...,1,0,0xdc6ff44d5d932cbd77b52e5612ba0529dc6226f1,800000000000000000,800000000000000000,0xf89d7b9c864f589bbf53a82105107622b35eaa40,0x73981e74c1b3d94cbe97e2cd03691dd2e7c533fa
1,2024-11-18,op,10,mainnet,1731890755,128145989,0x98f6d6cfb9de5b5ccf9c3d9849bc04ab9a2c4725b657...,0x026e632a5d3a57e54c0f786ae4249b318dcc9ff728f5...,4,1,0x4200000000000000000000000000000000000042,<NA>,238897558498147901946,0x4dc22588ade05c40338a9d95a6da9dcee68bcd60,0x802b65b5d9016621e66003aed0b16615093f328b
2,2024-11-18,op,10,mainnet,1731890755,128145989,0x98f6d6cfb9de5b5ccf9c3d9849bc04ab9a2c4725b657...,0x0f3da35693644abe74388ab718881855a50e147201bb...,6,10,0x4200000000000000000000000000000000000006,11330523516624,11330523516624,0x88391365c225973032275db256b9d15f845d2c72,0x07169e885228d41d986a5b66130ac6e3d8f44324
3,2024-11-18,op,10,mainnet,1731890755,128145989,0x98f6d6cfb9de5b5ccf9c3d9849bc04ab9a2c4725b657...,0x0f3da35693644abe74388ab718881855a50e147201bb...,6,14,0x4200000000000000000000000000000000000042,6755862736030762399,6755862736030762399,0x442659a6d04b907c879032da1ef634548110dd37,0xfc1f3296458f9b2a27a0b91dd7681c4020e09d05
4,2024-11-18,op,10,mainnet,1731890755,128145989,0x98f6d6cfb9de5b5ccf9c3d9849bc04ab9a2c4725b657...,0x026e632a5d3a57e54c0f786ae4249b318dcc9ff728f5...,4,2,0x4200000000000000000000000000000000000006,133713384334713000,133713384334713000,0x802b65b5d9016621e66003aed0b16615093f328b,0x4dc22588ade05c40338a9d95a6da9dcee68bcd60


In [10]:
erc721_transfers = (
    all_transfers
    .filter("token_id IS NOT NULL")
    .project("* EXCLUDE (amount, amount_lossless)")
)
df = ctx.client.sql(f"SELECT * FROM erc721_transfers").df()
df.head()

,dt,chain,chain_id,network,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,log_index,contract_address,from_address,to_address,token_id
0,2024-11-18,op,10,mainnet,1731890757,128145990,0x1dbc3f2bc6e28592c242c2da70c30d75cda987cee20f...,0x8c8097be6a11606a35153d59a846dd0563f461c67a2a...,8,38,0x416b433906b1b72fa758e166e239c43d68dc6f29,0x04f0d809f769772138736a1c79daf6b0a7692bb6,0xca573537505cce0d03963002e870d74d2cfe8cff,554014
1,2024-11-18,op,10,mainnet,1731890759,128145991,0xc88a780a0e73dda749ad445dc0a79ea40004b8863bbe...,0x0d3b8862452f1c602e247818e4b0b17ee71da7fb67ef...,32,115,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,0x0000000000000000000000000000000000000000,0xb7659fb7387584a337163a515e95022c1ab40553,3808461
2,2024-11-18,op,10,mainnet,1731890767,128145995,0x4fdd2eb67cf6a125a756fc73d699bc50bcf67cb5b519...,0xcfa408dd1654af2fbbffdec4a3411a857118ca58d567...,11,61,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,0x0000000000000000000000000000000000000000,0x9be0b20a1eb1065b49f0a908fbb705bbc25f2aa4,3808462
3,2024-11-18,op,10,mainnet,1731890771,128145997,0x4b93310e7e2306a99be902e4f5d2482c9e558a9f315f...,0x5b7fa54141974a14be23a1fb12f301c7439439145230...,24,53,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,0x0000000000000000000000000000000000000000,0xc3cdb79d301c097640657274f00c8adcb8016a84,3808463
4,2024-11-18,op,10,mainnet,1731890773,128145998,0x6b6d0b337b0c783371bf8d2af01f3d126386c82f5639...,0x820b6e0b0939c0954deb3dee3291c7aafa76204af6e4...,9,19,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,0x0000000000000000000000000000000000000000,0x8f37fdd3037b29195975d9e2f7bbb36ca51887dc,3808464


In [11]:
native_transfers = (
    native_transfers
    .filter("transfer_type = 'native'")
    .project("*")
)
df = ctx.client.sql(f"SELECT * FROM native_transfers").df()
df.head()

,dt,chain,chain_id,network,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,trace_address,from_address,to_address,amount,amount_lossless,input_method_id,call_type,transfer_type
0,2024-11-18,op,10,mainnet,1731888081,128144652,0xd5c05d3c61d74371e9c4461f8abf5aa8ab6319abf797...,0xcd629832d8c56f057f2b504630cc8de1ed4b1888a4c7...,2,"0,10,0,0",0xd2e2f6ffa2f282d91ca7b46811a82291a7e30fc1,0x63b122644d141cc2fafa8070c76d96b9c63f0fa1,18363421371027,18363421371027,0x,call,native
1,2024-11-18,op,10,mainnet,1731888083,128144653,0x0f6930a9b09ffb1d445222d49d67cada5b1c36ab5a2c...,0x9532b90f76ca299f8b9e2c57a693fad1181afe8e547e...,12,,0xe42870abb8f5453bf9c24acd56719c590e117910,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,75000000000000,75000000000000,0xd123b4d8,call,native
2,2024-11-18,op,10,mainnet,1731888085,128144654,0xf3539875e39b9a987b6f22a6b62faf5e113933cd5b9f...,0x6d238f83f6104a1674b08bac9c979cfb68da6baffa93...,5,,0xe773397e066b31e1fd413cf9a8b42c436fd462b5,0x1d46fcd1800cb35660137211b5db12d2696e4cd0,260459981433175,260459981433175,0x,call,native
3,2024-11-18,op,10,mainnet,1731888085,128144654,0xf3539875e39b9a987b6f22a6b62faf5e113933cd5b9f...,0x295474dd954c361d791cf64e349408fbc3b34b923425...,9,,0xdd50676f81f607fd8ba7ed3187ddf172db174cd3,0x5523d3c98809dddb82c686e152f5c58b1b0fb59e,19998030723480981,19998030723480981,0x8f0d6f17,call,native
4,2024-11-18,op,10,mainnet,1731888085,128144654,0xf3539875e39b9a987b6f22a6b62faf5e113933cd5b9f...,0x295474dd954c361d791cf64e349408fbc3b34b923425...,9,0,0x5523d3c98809dddb82c686e152f5c58b1b0fb59e,0x62f4e8cfd14962f0048ea77b3d99daa3d8d53cfd,19998030723480981,19998030723480981,0x,call,native
